# logging

## Logging `basicConfig`

The logging configuration can be set using the [`basicConfig`](https://docs.python.org/3/library/logging.html#logging.basicConfig) method. The most important arguments to provide are:
* `filename`: Specifies that a `FileHandler` be created, using the specified filename, rather than a `StreamHandler`, i.e. the logging messages will be printed in a log file instead of the console.
* `level`: Set the `root` logger level to the specified level. Logging levels will be discussed in the next section.
* `format`: Use the specified format string for the handler. Defaults to attributes `levelname`, `name` and `message` separated by colons. For more info consult the [LogRecord attributes documentation](https://docs.python.org/3/library/logging.html#logrecord-attributes).
* `datefmt`: Use the specified date/time format, as accepted by `time.strftime()`. You can read more about date/time formats in [link1](https://docs.python.org/3/library/time.html#time.strftime) or [link2](https://docs.python.org/3/library/datetime.html#format-codes).
* `filemode`: Specifies the write mode, where the default file mode is `a`.

In the code cell below, we set the `root` logger to:
* write the log messages in a file called `logging.log`
* only write logs with level `debug` and above.
* the log format should contain: the timestamp `asctime`, the logger's `name`, the `levelname`, and the `message` to be logged.
* the timestamp `asctime` should have a format of `YYYY-MM-DD (HH:MM:SS)`

In [1]:
import logging

logging.basicConfig(filename='root.log',
                    level=logging.DEBUG,
                    format='%(asctime)s: %(name)s: %(levelname)s: %(message)s',
                    datefmt='%Y-%m-%d (%H:%M:%S)')

After setting the `basicConfig`, the logged messages have the following format:

`2023-12-08 (09:30:51): root: INFO: This is an info message`

**N.B.** it is important to note that once the `root` config has been set using `basicConfig`, it cannot be changed by calling `basicConfig` again. Instead, each setting should be changed independantly by calling its corresponding setter.

For example, if we want to change the `root` `level` to `WARNING`, then we must use the following code snipet:

`logging.root.setLevel(logging.WARNING)`

Generally, it is a better practice to create various logger objects to handle the logs (will be discussed later). However, setting some basic formatting, `format` and `datefmt`, to the `root` logger can be beneficial, since logger objects can always fallback to the `root` logger settings if certain configs were not specifically set for the logger objects.

## Logging Levels

Logging encompasses **5** [logging levels](https://docs.python.org/3/library/logging.html#logging-levels), listed in ascending order of severity:
1. `debug`
2. `info`
3. `warning`
4. `error`
5. `critical`

We can add to those `exception` which logs a message with level `error` but with `Exception` info is added to the logging message.

The log messages are called likeso,

In [4]:
logging.debug('This is a debug message')
logging.info('This is an info message')
logging.warning('This is a warning message')
logging.error('This is an error message')
logging.critical('This is a critical message')
logging.exception('This is an exception message')

To highlight the difference between `error` and `exception`, we will log the message of a `ZeroDivisionError` using both methods.

In [3]:
try: 1/0
except ZeroDivisionError:
    logging.error(ZeroDivisionError.__name__)
    logging.exception(ZeroDivisionError.__name__)

The log file shows the following:

![`error` vs `exception`](./images/logging-error-vs-exception.png)

We remark that `exception` logged the `ZeroDivisionError` traceback along with the `message`, while `error` only logged the `message`.


It is important to note that when setting the logging level, the levels should be set using **capital** letters, like the code cell below.

For more infor refer to the [documentation](https://docs.python.org/3/library/logging.html#logging-levels).

In [ ]:
logging.root.setLevel(logging.NOTSET)
logging.root.setLevel(logging.DEBUG)
logging.root.setLevel(logging.INFO)
logging.root.setLevel(logging.WARNING)
logging.root.setLevel(logging.ERROR)
logging.root.setLevel(logging.CRITICAL)

## Logger Objects

It is advisable to adopt a preferable practice of constructing logger objects designed to manage logging in a distinct manner. Direct instantiation of loggers is discouraged; instead, it is recommended to utilize the module-level function `logging.getLogger(name)`. Repeated calls to `getLogger()` with the same name consistently yield a reference to the same Logger object.

In the code cell provided below, an instance of a logger object is instantiated with the assigned name 'tutorial-logger,' and its logging level is configured to `DEBUG`.

In [1]:
import logging

logger = logging.getLogger('tutorial-logger')
# set the log level for logger1
logger.setLevel(logging.DEBUG)

Now, attempting to print both an `info` and a `critical` message using the `logger`.

In [2]:
logger.info('info message from logger')
logger.critical('critical message from logger')

critical message from logger


In [3]:
logging.root.hasHandlers()

False

Observing two aspects:
1. The `info` message was not printed despite specifying the log level as `DEBUG`.
2. The message lacks formatting.

This occurs because we must define `handlers` and `formatters` for our `logger`.

It's essential to highlight that we haven't configured `basicConfig` for our `root`, causing the `logger` to lack fallback settings to the `root`.

By invoking `logging.basicConfig()` even with empty arguments, a `root` handler is established, enabling the `logger` to fallback to it.

We can confirm the creation of the root handler by invoking `logging.root.hasHandlers()`.

In [7]:
logging.basicConfig()

logger.info('info message from logger')
logger.critical('critical message from logger')

INFO:tutorial-logger:info message from logger
CRITICAL:tutorial-logger:critical message from logger


In [5]:
logging.root.hasHandlers()

True

Now, let's generate file handlers and formatters to assign to our logger.

In [2]:
# create a file_handler for logger
file_handler = logging.FileHandler('tutorial.log')
# create a formatter for the file_handler
formatter = logging.Formatter(fmt='%(asctime)s: %(name)s: %(levelname)s: %(message)s',
                              datefmt='%Y-%m-%d (%H:%M:%S)')
# and add it to the file_handler
file_handler.setFormatter(formatter)
# add the file_handler to the logger
logger.addHandler(file_handler)

Now, let's reprint both an `info` and a `critical` message using the configured `logger`.

In [8]:
logger.info('info message from logger')
logger.critical('critical message from logger')

INFO:tutorial-logger:info message from logger
CRITICAL:tutorial-logger:critical message from logger


Upon inspecting the tutorial-logger.log file, it's evident that the logger has correctly printed the logs in the desired format. However, the logger also displayed content in the stream, utilizing the default format of the `root`. This is attributed to the absence of a specified stream handler for the `logger`, prompting it to resort to the `root`'s stream handler.

To address this, we have two options:
1. Clear the root's stream handler if the intention is to avoid any stream output.
2. Create a custom stream handler for the `logger` to control the stream output as per our requirements.

Firstly, let's attempt option one by clearing the `root`'s stream handler.

In [9]:
logging.root.handlers.clear()
logging.root.hasHandlers()

False

Now, let's print both an `info` and a `critical` message again to observe the impact of clearing the root's stream handler.

In [10]:
logger.info('info message from logger')
logger.critical('critical message from logger')

Indeed, the `logger` only logged the messages to tutorial.log.

By configuring the `logger` and its `file_handler` before logging any messages, the log will also solely be directed to the file, eliminating streaming messages.

Now, let's create a stream handler to facilitate streaming messages through the console.

In [12]:
formatter = logging.Formatter(fmt='%(asctime)s.%(msecs)d: %(name)s: %(levelname)s: %(message)s',
                              datefmt='%Y-%m-%d %H:%M:%S')

stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.WARNING)
stream_handler.setFormatter(formatter)

logger.addHandler(stream_handler)

We can inspect the handlers associated with the logger by executing `logger.handlers`.

In [13]:
logger.handlers

[<FileHandler c:\Users\user\Desktop\MOOCs\python-tutorials\tutorial.log (NOTSET)>,
 <StreamHandler stderr (WARNING)>]

Certainly, it's evident from the output that we have two handlers configured for the logger: a file handler with a level of `NOTSET` and a stream handler with a level of `WARNING`. This setup allows for both file logging and streaming, with the specified levels indicating the severity threshold for each handler.

Now, let's print both an `info` and a `critical` message again to observe the impact of having both a file handler and a stream handler configured for the logger.

In [14]:
logger.info('info message from logger')
logger.critical('critical message from logger')

2023-12-08 20:22:32.76: tutorial-logger: CRITICAL: critical message from logger


Certainly, the observed behavior aligns with the configured settings. The stream handler, with a level of `WARNING`, printed only the critical message, as messages below this severity level were filtered out. On the other hand, the file handler, with a level of `NOTSET`, printed all messages, as it falls back to the logger's level, which is `DEBUG`. This provides a clear illustration of how handlers and their levels influence the logging output.

The order of fallback is a key aspect in understanding the logging hierarchy:

1. **Handler Fallback to Logger:**
   If a handler lacks a specific configuration, it falls back to the logger's configuration.

2. **Logger Fallback to Root Logger:**
   If the logger itself doesn't have a particular configuration, it falls back to the root logger's configuration.

This hierarchy allows for a flexible and cascading setup, ensuring that each component in the logging process can be configured at different levels, providing a balance between specificity and inheritance.

We can remove handlers from the logger using the `removeHandler` method.

In [19]:
print(logger.handlers)
logger.removeHandler(file_handler)
print(logger.handlers)

[<FileHandler c:\Users\user\Desktop\MOOCs\python-tutorials\tutorial.log (NOTSET)>, <StreamHandler stderr (WARNING)>]
[<StreamHandler stderr (WARNING)>]


To get the level of loggers and handlers, we can use

In [32]:
print(logger.level)
print(stream_handler.level)
logging.getLevelName(logger.level)

10
10


'DEBUG'

## `SysLogHandler`

The SysLogHandler class, located in the logging.handlers module, supports sending logging messages to a remote or local Unix syslog. In the following, we will just provide the synthax to set the handler.

In [ ]:
import logging
from logging.handlers import SysLogHandler

HOST = "xxx.yyy.com"
PORT = 12345

logger = logging.getLogger('remote-handler')
logger.setLevel(logging.INFO)
handler = SysLogHandler(address=(HOST, PORT))
handler.setLevel(logging.WARNING)
logger.addHandler(handler)

## `logging_tree`

The logging_tree module provides a hierarchical and graphical representation of the logging configuration in Python. When invoked, it displays a structured tree diagram illustrating the relationships between loggers and handlers, offering a comprehensive overview of the logging architecture. This tool proves invaluable for debugging and optimizing logging setups, enabling developers to easily visualize and comprehend the flow of log messages through different parts of their application.

In [15]:
from logging_tree import printout

printout()

<--""
   Level WARNING
   |
   o<--"Comm"
   |   Level NOTSET so inherits level WARNING
   |
   o   "IPKernelApp"
   |   Level DEBUG
   |   Propagate OFF
   |   Handler Stream <_io.TextIOWrapper name='<stderr>' mode='w' encoding='utf-8'>
   |     Level WARNING
   |     Formatter <traitlets.config.application.LevelFormatter object at 0x0000016BBFD7A710>
   |
   o<--"asyncio"
   |   Level NOTSET so inherits level WARNING
   |
   o<--[concurrent]
   |   |
   |   o<--"concurrent.futures"
   |       Level NOTSET so inherits level WARNING
   |
   o<--[ipykernel]
   |   |
   |   o<--"ipykernel.comm"
   |       Level NOTSET so inherits level WARNING
   |
   o<--[parso]
   |   |
   |   o<--"parso.cache"
   |   |   Level NOTSET so inherits level WARNING
   |   |
   |   o<--[parso.python]
   |       |
   |       o<--"parso.python.diff"
   |           Level NOTSET so inherits level WARNING
   |
   o<--[pkg_resources]
   |   |
   |   o<--[pkg_resources.extern]
   |       |
   |       o<--[pkg_resou